<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# EWHA bagging (3) - a

In [1]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()

file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [2]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
#query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

## 1. build DB

In [4]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os

UPSTAGE_API_KEY = api_key

layzer = UpstageLayoutAnalysisLoader(api_key=UPSTAGE_API_KEY,file_path=os.path.join(data_path, 'ewha.pdf'), output_type="text")

docs = layzer.load()  # or layzer.lazy_load()

/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis.py:135: UserWarning: UpstageLayoutAnalysisLoader is deprecated.Please use langchain_upstage.document_parse.UpstageDocumentParseLoader instead.
  warnings.warn(
/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis_parsers.py:160: UserWarning: UpstageLayoutAnalysisParser is deprecated.Please use langchain_upstage.document_parse_parsers.UpstageDocumentParseParser instead.
  warnings.warn(


## 2. Text Split

In [5]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

chunk_size, chunk_overlap = 500, 50

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

Splits: 106


In [6]:
print(splits[0])
len(splits[0].page_content)

page_content='이화여자대학교 학칙1946. 8. 15. 제정
2017. 8. 16. 개정제1장 총칙제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.제2조(명칭) 본교는 이화여자대학교라 부른다.
제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)제2장 편제제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원' metadata={'total_pages': 54}


500

In [11]:
chunk = []
for index in range(0,len(splits)) : # context 받아오기
    chunk.append(splits[index].page_content)

embedded_documents = passage_embeddings.embed_documents(chunk)

In [12]:
import numpy as np
np.save(data_path+f'embedding/full_ewha{chunk_size}', np.array(chunk))
np.save(data_path+f'embedding/full_ewha{chunk_size}_embed', np.array(embedded_documents))